<a href="https://colab.research.google.com/github/dcsan/dalle-playground/blob/dc/v2/backend/render_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[DC fork v3.0](https://github.com/dcsan/dalle-playground)

### <center>Use this notebook to run your DALL-E server</center>
### <center> [DALL-E Playground Repository](https://github.com/saharmor/dalle-playground) </center>


#####<center>This notebook is based on the amazing [DALL·E Mini](https://github.com/borisdayma/dalle-mini) by [Boris Dayma](https://github.com/borisdayma) et al.</center>

## Install dependencies

In [12]:
!rm -rf dalle-playground/
!git clone https://github.com/saharmor/dalle-playground.git
!pip install -r dalle-playground/backend/requirements.txt
!pip install flask-cloudflared

Cloning into 'dalle-playground'...
remote: Enumerating objects: 479, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 479 (delta 11), reused 24 (delta 6), pack-reused 445
Receiving objects: 100% (479/479), 1.50 MiB | 7.25 MiB/s, done.
Resolving deltas: 100% (269/269), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-93xcytmt
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-93xcytmt
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/patil-suraj/vqgan-jax.git to /tmp/pip-req-build-rz1cu2z1
  Running command git clone -q https://github.com/patil-suraj/vqgan-jax.git /tmp/pip-req-build-rz1cu2z1
  Cloning https://github.com/borisdayma/dalle-mi

## Choose a model

* **DALL-E Mini**
  * The original DALL-E Mini. Fastest yet suboptimal results
  * Runs well on a 4GB GPU and the Google Colab free tier
* **DALL-E Mega**
  * The advanced version of DALL-E Mini. Requires more compute and VRAM
  * Runs well on a 8GB GPU and recommended on the Google Colab Pro tier
* **DALL-E Mega Full**
  * The most performant DALL-E version. Requires significantly more computre and VRAM
  * Runs well on a 12GB GPU and recommended on the Google Colab Pro+ tier


In [13]:
# check run time type to see if we have a GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jun 14 21:40:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
# check TPU
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  print('not connected to TPU')
  # raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')


Tensorflow version 2.8.2
not connected to TPU


In [16]:
#@title Select a model - remember to execute cell after selecting!

dalle_model = 'Mini' #@param ["Mini", "Mega", "Mega_full"]

# Run the backend web server

In [ ]:
from threading import Thread, Event
import requests

app_port = 8000
announce_url = None
cloudflared_startup = Event()

def post_url(url):
  update_url = 'http://revel.alfalabs.xyz/api/revel/imagenServer/set'
  json = {'url': url}
  result = requests.post(update_url, json=json)
  print('posted', result)


def update_announce_url(url):
    global announce_url
    announce_url = url

def start_cloudflared(port):
  from flask_cloudflared import _run_cloudflared
  try:
    announce_url = _run_cloudflared(port)
  except:
    raise
  finally:
    update_announce_url(announce_url)
    cloudflared_startup.set()

def run_with_cloudflared(thread):
    old_run = thread.run

    def new_run(*args, **kwargs):
        new_thread = Thread(target = start_cloudflared, args=(app_port,))
        new_thread.setDaemon(True)
        new_thread.start()
        old_run(*args, **kwargs)

    thread.run = new_run

def app():
  print(f"Selected DALL-E Model - [{dalle_model}]")
  !python dalle-playground/backend/app.py --port {app_port} --model_version {dalle_model} --save_to_disk false

if __name__ == '__main__':
    t1 = Thread(target = app)
    run_with_cloudflared(t1)
    t1.start()
    cloudflared_startup.wait()
    print(f"Your url is: {announce_url}")
    post_url(announce_url)
    t1.join()

Selected DALL-E Model - [Mega]
--> Starting DALL-E Server. This might take up to two minutes.
your url is: https://bumpy-taxis-shave-104-199-131-126.loca.lt
tcmalloc: large alloc 5175050240 bytes == 0xbac2000 @  0x7fd7f27201e7 0x4a3940 0x5b438c 0x5ea94f 0x5939cb 0x594cd3 0x5d0ecb 0x59aeca 0x515655 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549e0e 0x593fce 0x548ae9 0x5127f1 0x4bc98a 0x532b86 0x594a96 0x515600 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7fd7f231dc87 0x5b621a
Some of the weights of DalleBart were initialized in float16 precision from the model checkpoint at /tmp/tmpe3q0je1f:
[('lm_head', 'kernel'), ('model', 'decoder', 'embed_positions', 'embedding'), ('model', 'decoder', 'embed_tokens', 'embedding'), ('model', 'decoder', 'final_ln', 'bias'), ('model', 'decoder', 'layernorm_embedding', 'bias'), ('model', 'decoder', 'layernorm_embedding', 'scale'), ('model', 'decoder', 'layers', 'FlaxBartDecoderLayers', 'FlaxBartAttention_0', 'k_proj', 'kernel'), (

### Now, take the url you got from cloudflare (should look like `your url is: https://xxxxxx.trycloudflare.com`), replace it within the url here https://saharmor.github.io/dalle-playground/?backendUrl=https://xxxxxx.trycloudflare.com and run it in the browser.

### Let the fun begin ✨